In [1]:
import tensorflow as tf
import tensorflow_addons as tfa
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers

C:\Users\PC\anaconda3\lib\site-packages\tensorflow_addons\utils\tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


# Data preparation

In [7]:
import pandas as pd
def data_preparation():
    
    data1 = pd.read_csv("path/to/data1.csv")
    data2 = pd.read_csv("path/to/data2.csv")
  
    
    print('data1', data1.shape)
    print('data2', data2.shape)
       
    return data1 , data2

In [8]:
def split_df(df, train_size=0.8):
    """Splits a Pandas DataFrame into train and validation sets."""
    n = len(df)
    
    # shufling
    df = df.sample(frac=1, random_state=0)
    
    split_index = int(n * train_size)
    train_df = df.iloc[:split_index]
    validation_df = df.iloc[split_index:]
        
    train_x = train_df.iloc[:, 0:8].to_numpy()
    train_y = train_df.iloc[:, -1].to_numpy()
    
    validation_x = validation_df.iloc[:, 0:8].to_numpy()
    validation_y = validation_df.iloc[:, -1].to_numpy()
    
    return train_x, validation_x, train_y, validation_y


# Data normalisation and EDA are Required (very importatne) .......................................................................













# Data Preparation

In [9]:
"""
## Prepare the data
"""
#####################################################################
data1 , data2 = data_preparation()
#########################################################################
# Select to source domaine !
source_domain =  data1
x_train, x_test, y_train, y_test = split_df(source_domain)
#########################################################################
print(f"x_train shape: {x_train.shape} - y_train shape: {y_train.shape}")
print(f"x_test shape: {x_test.shape} - y_test shape: {y_test.shape}")

data1 (10000, 10)
data2 (10000, 10)
x_train shape: (8000, 8) - y_train shape: (8000,)
x_test shape: (2000, 8) - y_test shape: (2000,)


# Training binary classification on the source domain

## Build the classification model (encoder model)

In [10]:
"""
## Using image data normalization layer ( can be used also for data augmentation)
"""
data_augmentation = keras.Sequential(
    [
        layers.Normalization(),
        # layers.RandomFlip("horizontal"),
        # layers.RandomRotation(0.02),
    ]
)

# Setting the state of the normalization layer.
data_augmentation.layers[0].adapt(x_train)

##########################################################################################
"""
## Build the encoder model

The encoder model takes the image as input and turns it into a 8-dimensional feature vector.
"""

# cnn model
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
from matplotlib import pyplot
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.utils import to_categorical
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dropout, Dense, Input


num_classes = 1
learning_rate = 0.001
batch_size = 128 
hidden_units = 256
projection_units = 32
num_epochs = 100
dropout_rate = 0.5
temperature = 0.05
n_features = 8
verbose, epochs = 0, 50

x_train = np.expand_dims(x_train, axis=-1)
print('x_train shape ', x_train.shape)

########################################################################
# # fit and evaluate a model
def conv1D_model(trainX, trainy, testX, testy):
    # verbose, epochs = 0, 10
    n_features = trainX.shape[1]
    
    model = Sequential()
    model.add(Conv1D(filters=64, 
                     kernel_size=3, 
                     activation='relu', 
                     input_shape=(n_features, 1)
                    ))
    model.add(Conv1D(filters=64, 
                     kernel_size=3, 
                     activation='relu'))
    
    model.add(Dropout(0.5))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(8, activation='relu')) 
    # model.summary()
    return model
  
####################################################################
def create_encoder():
    
    conv_model = conv1D_model(x_train, y_train, x_test, y_test)
    inputs = keras.Input(shape=(n_features, ))
    augmented = data_augmentation(inputs)
    outputs   = conv_model(augmented)
    model     = keras.Model(inputs=inputs, 
                        outputs=outputs, 
                        name="encoder"
                       )
    return model

######################################################################
encoder = create_encoder()
encoder.summary()
######################################################################

## Build the classification model

def create_classifier(encoder, trainable=True):
    for layer in encoder.layers:
        layer.trainable = trainable

    inputs   = keras.Input(shape=(n_features, ))
    features = encoder(inputs)
    features = layers.Dense(hidden_units, activation="relu")(features)
    features = layers.Dropout(dropout_rate)(features)
    features = layers.Dense(32, activation="relu")(features)
    outputs = layers.Dense(num_classes, activation="sigmoid")(features)

    model = keras.Model(inputs=inputs, outputs=outputs, name="classifier")
    model.compile(loss='binary_crossentropy', 
                  optimizer='adam', 
                  metrics=['accuracy']
                 )
    return model

x_train shape  (8000, 8, 1)
Model: "encoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 8)]               0         
                                                                 
 sequential (Sequential)     (None, 8)                 17        
                                                                 
 sequential_1 (Sequential)   (None, 8)                 17000     
                                                                 
Total params: 17017 (66.48 KB)
Trainable params: 17000 (66.41 KB)
Non-trainable params: 17 (72.00 Byte)
_________________________________________________________________


In [11]:

encoder = create_encoder()
classifier = create_classifier(encoder)
classifier.summary()






Model: "classifier"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 8)]               0         
                                                                 
 encoder (Functional)        (None, 8)                 17017     
                                                                 
 dense_4 (Dense)             (None, 256)               2304      
                                                                 
 dropout_4 (Dropout)         (None, 256)               0         
                                                                 
 dense_5 (Dense)             (None, 32)                8224      
                                                                 
 dense_6 (Dense)             (None, 1)                 33        
                                                                 
Total params: 27578 (107.73 KB)
Trainable params: 27561 

In [12]:
encoder.summary()


Model: "encoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 8)]               0         
                                                                 
 sequential (Sequential)     (None, 8)                 17        
                                                                 
 sequential_2 (Sequential)   (None, 8)                 17000     
                                                                 
Total params: 17017 (66.48 KB)
Trainable params: 17000 (66.41 KB)
Non-trainable params: 17 (72.00 Byte)
_________________________________________________________________


## Train the baseline classification model
In this experiment, a baseline classifier is trained as usual, i.e., the
encoder and the classifier parts are trained together as a single model
to minimize the crossentropy loss.


In [13]:
encoder = create_encoder()
classifier = create_classifier(encoder)

classifier.summary()

history = classifier.fit(x=x_train, 
                         y=y_train, 
                         batch_size=batch_size, 
                         epochs=num_epochs
                        )

accuracy = classifier.evaluate(x_test, y_test)[1]
print(f"Test accuracy: {round(accuracy * 100, 2)}%")


Model: "classifier"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 8)]               0         
                                                                 
 encoder (Functional)        (None, 8)                 17017     
                                                                 
 dense_9 (Dense)             (None, 256)               2304      
                                                                 
 dropout_7 (Dropout)         (None, 256)               0         
                                                                 
 dense_10 (Dense)            (None, 32)                8224      
                                                                 
 dense_11 (Dense)            (None, 1)                 33        
                                                                 
Total params: 27578 (107.73 KB)
Trainable params: 27561 

63/63 [==============================] - 0s 3ms/step - loss: 0.0097 - accuracy: 0.9974
Epoch 71/100
63/63 [==============================] - 0s 3ms/step - loss: 0.0120 - accuracy: 0.9961
Epoch 72/100
63/63 [==============================] - 0s 3ms/step - loss: 0.0088 - accuracy: 0.9971
Epoch 73/100
63/63 [==============================] - 0s 3ms/step - loss: 0.0087 - accuracy: 0.9973
Epoch 74/100
63/63 [==============================] - 0s 3ms/step - loss: 0.0059 - accuracy: 0.9980
Epoch 75/100
63/63 [==============================] - 0s 3ms/step - loss: 0.0118 - accuracy: 0.9965
Epoch 76/100
63/63 [==============================] - 0s 3ms/step - loss: 0.0099 - accuracy: 0.9974
Epoch 77/100
63/63 [==============================] - 0s 3ms/step - loss: 0.0123 - accuracy: 0.9964
Epoch 78/100
63/63 [==============================] - 0s 3ms/step - loss: 0.0081 - accuracy: 0.9977
Epoch 79/100
63/63 [==============================] - 0s 3ms/step - loss: 0.0067 - accuracy: 0.9977
Epoch 80/100


# Transfer Learning

In [15]:
# Select the target domain 
# shufling and select a small dataset
target_domain = data2
target_domain = target_domain.sample(frac=1, random_state=0)

# Select the desired train_size
x_train, x_test, y_train, y_test = split_df(target_domain, 
                                            train_size=0.001
                                           )
#########################################################################
print('x_train.shape :', x_train.shape)
print('x_test.shape :', x_test.shape)
#########################################################################

## Train the classifier with the frozen encoder
classifier = create_classifier(encoder, 
                               trainable=False
                              )

history = classifier.fit(x=x_train, 
                         y=y_train, 
                         batch_size=batch_size, 
                         epochs=num_epochs
                        )

accuracy = classifier.evaluate(x_test, y_test)[1]
print(f"Test accuracy: {round(accuracy * 100, 2)}%")

x_train.shape : (10, 8)
x_test.shape : (9990, 8)
Epoch 1/100
1/1 [==============================] - 1s 590ms/step - loss: 0.7056 - accuracy: 0.5000
Epoch 2/100
1/1 [==============================] - 0s 3ms/step - loss: 0.7342 - accuracy: 0.5000
Epoch 3/100
1/1 [==============================] - 0s 4ms/step - loss: 0.6279 - accuracy: 0.5000
Epoch 4/100
1/1 [==============================] - 0s 5ms/step - loss: 0.5698 - accuracy: 0.8000
Epoch 5/100
1/1 [==============================] - 0s 5ms/step - loss: 0.5123 - accuracy: 0.8000
Epoch 6/100
1/1 [==============================] - 0s 4ms/step - loss: 0.5953 - accuracy: 0.7000
Epoch 7/100
1/1 [==============================] - 0s 5ms/step - loss: 0.5354 - accuracy: 0.7000
Epoch 8/100
1/1 [==============================] - 0s 4ms/step - loss: 0.5286 - accuracy: 0.9000
Epoch 9/100
1/1 [==============================] - 0s 4ms/step - loss: 0.4614 - accuracy: 0.9000
Epoch 10/100
1/1 [==============================] - 0s 4ms/step - loss: 0.43

1/1 [==============================] - 0s 4ms/step - loss: 0.0248 - accuracy: 1.0000
Epoch 85/100
1/1 [==============================] - 0s 3ms/step - loss: 0.0103 - accuracy: 1.0000
Epoch 86/100
1/1 [==============================] - 0s 4ms/step - loss: 0.0396 - accuracy: 1.0000
Epoch 87/100
1/1 [==============================] - 0s 4ms/step - loss: 0.0136 - accuracy: 1.0000
Epoch 88/100
1/1 [==============================] - 0s 4ms/step - loss: 0.0164 - accuracy: 1.0000
Epoch 89/100
1/1 [==============================] - 0s 5ms/step - loss: 0.0177 - accuracy: 1.0000
Epoch 90/100
1/1 [==============================] - 0s 4ms/step - loss: 0.0029 - accuracy: 1.0000
Epoch 91/100
1/1 [==============================] - 0s 3ms/step - loss: 0.0118 - accuracy: 1.0000
Epoch 92/100
1/1 [==============================] - 0s 3ms/step - loss: 0.0055 - accuracy: 1.0000
Epoch 93/100
1/1 [==============================] - 0s 4ms/step - loss: 0.0160 - accuracy: 1.0000
Epoch 94/100
1/1 [===============

In [16]:
from sklearn.metrics import classification_report
y_pred=classifier.predict(x_test,batch_size=batch_size)
y_pred = np.squeeze(y_pred)
y_pred = np.where(y_pred < 0.5, 0, 1)



79/79 [==============================] - 0s 859us/step


In [17]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         0.0       0.99      0.97      0.98      4995
         1.0       0.97      0.99      0.98      4995

    accuracy                           0.98      9990
   macro avg       0.98      0.98      0.98      9990
weighted avg       0.98      0.98      0.98      9990

